# Введение в обработку естественного языка
## Урок 8. Рекуррентные нейронные сети RNN LSTM GRU

### Задание
Данные берем отызывы за лето
На вебинаре мы говорили, что долгое время CNN и RNN архитектуры были конурируещими выяснить какая архитектура больше подходит для нашей задачи
1. построить свёрточные архитектуры
2. построить различные архитектуры с RNN
3. построить совместные архитектуры CNN -> RNN или (RNN -> CNN)

In [1]:
!pip install --upgrade xlrd
!pip install pymorphy2
!pip install stop-words

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 96 kB 4.5 MB/s 
  Attempting uninstall: xlrd
    Found existing installation: xlrd 1.1.0
    Uninstalling xlrd-1.1.0:
      Successfully uninstalled xlrd-1.1.0
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 55 kB 2.7 MB/s 
     |████████████████████████████████| 8.2 MB 10.7 MB/s 
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Created wheel for stop-words: filename=stop_words-2018.7.23-py3-none-any.whl size=32911 sha256=8e21eb06608a2b4c75438b9de0bfb11801265a335f05a65bdfcc0cd84a9c1936
  Stored in directory: /root/.cache/pip/wheels/fb/86/b2/277b10b1ce9f73ce15059bf6975d4547cc4ec3feeb651978e9
Successfully built stop-words


In [8]:
import pandas as pd
from string import punctuation
from stop_words import get_stop_words
from pymorphy2 import MorphAnalyzer
import re
from sklearn.model_selection import train_test_split
import keras
from keras.models import Sequential, Model
from keras.layers import Dense, Dropout, Activation, Input, Embedding, Conv1D, GlobalMaxPool1D, SimpleRNN, LSTM, GRU, Masking, Flatten
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.callbacks import TensorBoard 
from keras.callbacks import EarlyStopping

In [3]:
df = pd.read_excel('/content/отзывы за лето.xls')
df.Content = df.Content.astype(str)

In [4]:
sw = set(get_stop_words("ru"))
exclude = set(punctuation)
morpher = MorphAnalyzer()

def preprocess_text(txt):
    txt = str(txt)
    txt = "".join(c for c in txt if c not in exclude)
    txt = txt.lower()
    txt = re.sub("не\s", "не", txt)
    txt = [morpher.parse(word)[0].normal_form for word in txt.split() if word not in sw]
    return " ".join(txt)

df['Content'] = df['Content'].apply(preprocess_text)
df.sample(5)

,Rating,Content,Date
17549,5,удобно оперативно,2017-07-11
4810,5,отличный работа,2017-08-07
3188,5,разработчик супер,2017-08-09
11663,1,приложение нравиться прийтись сбросить настрой...,2017-07-28
14904,4,😄😄😄,2017-07-22


In [6]:
train_df, test_val_df = train_test_split(df, test_size=0.3, random_state=49)
test_df, val_df = train_test_split(test_val_df, test_size=0.5, random_state=49)

text_corpus_train = train_df['Content'].values
text_corpus_valid = val_df['Content'].values
text_corpus_test = test_df['Content'].values

In [10]:
num_classes = len(df['Rating'].unique())
tokenizer = Tokenizer(num_words=None, 
                     filters='#$%&()*+-<=>@[\\]^_`{|}~\t\n',
                     lower = False, split = ' ')
tokenizer.fit_on_texts(text_corpus_train)

sequences_train = tokenizer.texts_to_sequences(text_corpus_train)
sequences_val = tokenizer.texts_to_sequences(text_corpus_valid)
sequences_test = tokenizer.texts_to_sequences(text_corpus_test)

word_count = len(tokenizer.index_word) + 1
training_length = max([len(i.split()) for i in text_corpus_train])

X_train = pad_sequences(sequences_train, maxlen=training_length)
X_valid = pad_sequences(sequences_val, maxlen=training_length)

y_train = keras.utils.np_utils.to_categorical(train_df['Rating'], num_classes+1)
y_test = keras.utils.np_utils.to_categorical(test_df['Rating'], num_classes+1)
y_val = keras.utils.np_utils.to_categorical(val_df['Rating'], num_classes+1)

### построить различные архитектуры с RNN

In [11]:
model = Sequential()

model.add(
    Embedding(input_dim=word_count,
              input_length=training_length,
              output_dim=30,
              trainable=True,
              mask_zero=True))
model.add(Masking(mask_value=0.0))

model.add(SimpleRNN(132))
model.add(Dense(132, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(num_classes+1, activation='softmax'))

model.compile(
    optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

early_stopping=EarlyStopping(monitor='val_loss',patience =2,restore_best_weights=1)

In [12]:
history = model.fit(X_train, y_train,
                    batch_size=512,
                    epochs=10,
                    verbose=1,
                    validation_split=0.1,
                    callbacks=[early_stopping])

Epoch 1/10
26/26 [==============================] - 15s 505ms/step - loss: 1.1210 - accuracy: 0.6855 - val_loss: 0.9227 - val_accuracy: 0.7291
Epoch 2/10
26/26 [==============================] - 13s 488ms/step - loss: 0.9929 - accuracy: 0.7007 - val_loss: 0.8119 - val_accuracy: 0.7374
Epoch 3/10
26/26 [==============================] - 15s 591ms/step - loss: 0.8184 - accuracy: 0.7341 - val_loss: 0.7148 - val_accuracy: 0.7616
Epoch 4/10
26/26 [==============================] - 14s 529ms/step - loss: 0.7167 - accuracy: 0.7593 - val_loss: 0.6808 - val_accuracy: 0.7713
Epoch 5/10
26/26 [==============================] - 13s 503ms/step - loss: 0.6457 - accuracy: 0.7795 - val_loss: 0.6537 - val_accuracy: 0.7761
Epoch 6/10
26/26 [==============================] - 13s 482ms/step - loss: 0.5917 - accuracy: 0.7928 - val_loss: 0.6563 - val_accuracy: 0.7719
Epoch 7/10
26/26 [==============================] - 13s 484ms/step - loss: 0.5494 - accuracy: 0.8091 - val_loss: 0.6683 - val_accuracy: 0.7754

In [13]:
score = model.evaluate(X_valid, y_val, batch_size=512, verbose=1)
print('Test score:', score[0])
print('Test accuracy:', score[1])

7/7 [==============================] - 1s 133ms/step - loss: 0.6809 - accuracy: 0.7631
Test score: 0.6809259057044983
Test accuracy: 0.7631493806838989


In [14]:
results = []
results.append(['RNN', score[0], score[1]])

In [17]:
model = Sequential()

model.add(
    Embedding(input_dim=word_count,
              input_length=training_length,
              output_dim=30,
              trainable=True,
              mask_zero=True))
model.add(Masking(mask_value=0.0))
model.add(LSTM(132, recurrent_dropout=0.2))
model.add(Dense(132, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(num_classes+1, activation='softmax'))

model.compile(
    optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

In [18]:
early_stopping=EarlyStopping(monitor='val_loss',patience =2, restore_best_weights=1)  

history = model.fit(X_train, y_train,
                    batch_size=512,
                    epochs=10,
                    verbose=1,
                    validation_split=0.1,
                    callbacks=[early_stopping])

Epoch 1/10
26/26 [==============================] - 56s 2s/step - loss: 1.2594 - accuracy: 0.6817 - val_loss: 0.8990 - val_accuracy: 0.7291
Epoch 2/10
26/26 [==============================] - 56s 2s/step - loss: 0.9158 - accuracy: 0.7062 - val_loss: 0.7688 - val_accuracy: 0.7346
Epoch 3/10
26/26 [==============================] - 49s 2s/step - loss: 0.7744 - accuracy: 0.7452 - val_loss: 0.6949 - val_accuracy: 0.7643
Epoch 4/10
26/26 [==============================] - 54s 2s/step - loss: 0.6849 - accuracy: 0.7678 - val_loss: 0.6705 - val_accuracy: 0.7713
Epoch 5/10
26/26 [==============================] - 54s 2s/step - loss: 0.6243 - accuracy: 0.7825 - val_loss: 0.6568 - val_accuracy: 0.7768
Epoch 6/10
26/26 [==============================] - 49s 2s/step - loss: 0.5840 - accuracy: 0.7935 - val_loss: 0.6549 - val_accuracy: 0.7747
Epoch 7/10
26/26 [==============================] - 49s 2s/step - loss: 0.5454 - accuracy: 0.8064 - val_loss: 0.6626 - val_accuracy: 0.7726
Epoch 8/10
26/26 [==

In [19]:
score = model.evaluate(X_valid, y_val, batch_size=512, verbose=1)
print('Test score:', score[0])
print('Test accuracy:', score[1])

results.append(['LSTM', score[0], score[1]])

7/7 [==============================] - 3s 388ms/step - loss: 0.6763 - accuracy: 0.7712


Test score: 0.6763240694999695
Test accuracy: 0.7712165117263794


In [20]:
model = Sequential()

model.add(
    Embedding(input_dim=word_count,
              input_length=training_length,
              output_dim=30,
              trainable=True,
              mask_zero=True))
model.add(Masking(mask_value=0.0))
model.add(GRU(64, recurrent_dropout=0.2))
model.add(Dense(64, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(num_classes+1, activation='softmax'))
model.compile(
    optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

model.summary()

Model: "sequential_3"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_3 (Embedding)     (None, 132, 30)           294870    
                                                                 
 masking_3 (Masking)         (None, 132, 30)           0         
                                                                 
 gru (GRU)                   (None, 64)                18432     
                                                                 
 dense_6 (Dense)             (None, 64)                4160      
                                                                 
 dropout_3 (Dropout)         (None, 64)                0         
                                                                 
 dense_7 (Dense)             (None, 6)                 390       
                                                                 
Total params: 317,852
Trainable params: 317,852
Non-tr

In [21]:
early_stopping=EarlyStopping(monitor='val_loss',patience =2, restore_best_weights=1)  


history = model.fit(X_train, y_train,
                    batch_size=512,
                    epochs=10,
                    verbose=1,
                    validation_split=0.1,
                    callbacks=[early_stopping])

Epoch 1/10
26/26 [==============================] - 18s 606ms/step - loss: 1.4513 - accuracy: 0.6714 - val_loss: 0.9709 - val_accuracy: 0.7291
Epoch 2/10
26/26 [==============================] - 15s 590ms/step - loss: 0.9444 - accuracy: 0.7052 - val_loss: 0.7923 - val_accuracy: 0.7291
Epoch 3/10
26/26 [==============================] - 15s 591ms/step - loss: 0.8237 - accuracy: 0.7084 - val_loss: 0.7418 - val_accuracy: 0.7415
Epoch 4/10
26/26 [==============================] - 15s 588ms/step - loss: 0.7528 - accuracy: 0.7423 - val_loss: 0.7087 - val_accuracy: 0.7671
Epoch 5/10
26/26 [==============================] - 15s 590ms/step - loss: 0.6760 - accuracy: 0.7741 - val_loss: 0.6845 - val_accuracy: 0.7692
Epoch 6/10
26/26 [==============================] - 15s 588ms/step - loss: 0.6168 - accuracy: 0.7892 - val_loss: 0.6660 - val_accuracy: 0.7754
Epoch 7/10
26/26 [==============================] - 16s 630ms/step - loss: 0.5757 - accuracy: 0.8010 - val_loss: 0.6757 - val_accuracy: 0.7733

In [22]:
score = model.evaluate(X_valid, y_val, batch_size=512, verbose=1)
print('Test score:', score[0])
print('Test accuracy:', score[1])

results.append(['GRU', score[0], score[1]])

7/7 [==============================] - 1s 107ms/step - loss: 0.6897 - accuracy: 0.7667
Test score: 0.6896584033966064
Test accuracy: 0.766698956489563


### построить совместные архитектуры CNN -> RNN или (RNN -> CNN)

In [23]:
model = Sequential()

model.add(
    Embedding(input_dim=word_count,
              input_length=training_length,
              output_dim=30,
              trainable=True,
              mask_zero=True))
model.add(SimpleRNN(132, recurrent_dropout=0.2, return_sequences="True"))
model.add(Conv1D(132, 3, activation="linear"))
model.add(Conv1D(64, 1, activation="linear")) 
model.add(Flatten())                      
model.add(Dropout(0.5)) 
model.add(Dense(num_classes+1, activation="softmax"))      


model.compile(
    optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

model.summary()

Model: "sequential_4"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_4 (Embedding)     (None, 132, 30)           294870    
                                                                 
 simple_rnn_1 (SimpleRNN)    (None, 132, 132)          21516     
                                                                 
 conv1d (Conv1D)             (None, 130, 132)          52404     
                                                                 
 conv1d_1 (Conv1D)           (None, 130, 64)           8512      
                                                                 
 flatten (Flatten)           (None, 8320)              0         
                                                                 
 dropout_4 (Dropout)         (None, 8320)              0         
                                                                 
 dense_8 (Dense)             (None, 6)                

In [24]:
early_stopping=EarlyStopping(monitor='val_loss',patience =2, restore_best_weights=1)  


history = model.fit(X_train, y_train,
                    batch_size=512,
                    epochs=10,
                    verbose=1,
                    validation_split=0.1,
                    callbacks=[early_stopping])

Epoch 1/10
26/26 [==============================] - 32s 1s/step - loss: 1.1314 - accuracy: 0.6910 - val_loss: 0.7938 - val_accuracy: 0.7554
Epoch 2/10
26/26 [==============================] - 30s 1s/step - loss: 0.8097 - accuracy: 0.7343 - val_loss: 0.7184 - val_accuracy: 0.7671
Epoch 3/10
26/26 [==============================] - 30s 1s/step - loss: 0.6864 - accuracy: 0.7636 - val_loss: 0.6978 - val_accuracy: 0.7768
Epoch 4/10
26/26 [==============================] - 30s 1s/step - loss: 0.5975 - accuracy: 0.7878 - val_loss: 0.6851 - val_accuracy: 0.7768
Epoch 5/10
26/26 [==============================] - 30s 1s/step - loss: 0.5339 - accuracy: 0.8074 - val_loss: 0.7076 - val_accuracy: 0.7726
Epoch 6/10
26/26 [==============================] - 30s 1s/step - loss: 0.4882 - accuracy: 0.8237 - val_loss: 0.7929 - val_accuracy: 0.7692


In [25]:
score = model.evaluate(X_valid, y_val, batch_size=512, verbose=1)
print('Test score:', score[0])
print('Test accuracy:', score[1])

results.append(['RNN + CNN', score[0], score[1]])

7/7 [==============================] - 2s 257ms/step - loss: 0.7144 - accuracy: 0.7635
Test score: 0.7144280076026917
Test accuracy: 0.7634720802307129


In [26]:
results_df = pd.DataFrame(results, columns = ['Model', 'Test score', 'Test accuracy'])
results_df

,Model,Test score,Test accuracy
0,RNN,0.680926,0.763149
1,LSTM,0.676324,0.771217
2,GRU,0.689658,0.766699
3,RNN + CNN,0.714428,0.763472
